## 0. Importing all necessary libs and tools

In [2]:
import torch
from ultralytics import YOLO
import os
import time
import sys
for module_name in list(sys.modules.keys()):
    if module_name.startswith('src.'):
        del sys.modules[module_name]
from src.config import MODEL_NAME, OVERLAPPING_PERCENTAGE, SLICE_SIZE, SLICES_FOLDER, PREDICT_FOLDER_PREFIX, CONFIDENCE_THRESHOLD, IOU_FOLDER_PREFIX, OUTLIER_FILTER_FOLDER_PREFIX, OUTLIER_THRESHOLD_K, IOU_THRESHOLD
print(MODEL_NAME, SLICE_SIZE, SLICES_FOLDER, PREDICT_FOLDER_PREFIX, CONFIDENCE_THRESHOLD, IOU_FOLDER_PREFIX, OUTLIER_FILTER_FOLDER_PREFIX, OUTLIER_THRESHOLD_K, IOU_THRESHOLD)
from src.model import get_devices, get_models, process_images, read_annotation_file, create_annotation_file
from src.tools.utils import get_destination_folder, create_destination_folder, get_slice_coordinates
from src.tools.image_slicer import create_image_slices


best_4 512 slices predicted_images_with_annotations 0.4 iou_filtered outlier_filtered 3 0.5


## 1. Slicing image process


In [ ]:
images_folder = "dataset/0.0.1/АФС для обработки ИИ"
image_paths = [f"{images_folder}/{image_path}" for image_path in os.listdir(images_folder)]
for image_path in image_paths:
    create_image_slices(
        image_path=image_path, 
        overlap_percentage=OVERLAPPING_PERCENTAGE, 
        destination_folder=SLICES_FOLDER,
        slice_size=SLICE_SIZE
    )

## 1.2 Parallel slicing approach

In [ ]:
from src.tools.image_slicer import create_image_slices_parallel
create_image_slices_parallel(OVERLAPPING_PERCENTAGE, SLICES_FOLDER, SLICE_SIZE)

Processing 211 images using 3 processes


Processing images:   0%|          | 0/211 [00:00<?, ?it/s]

## 2. Processing slices with yolo model 

In [ ]:
devices = get_devices()
models = get_models(MODEL_NAME, devices)
folder_name = get_destination_folder([PREDICT_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD])
output_folder = create_destination_folder(folder_name)
boxes_list = process_images(models[0], output_folder, devices[0])